In [31]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn import decomposition
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.multioutput import MultiOutputRegressor
from matplotlib import pyplot as plt
import matplotlib.style as style
import time

In [30]:
import plotly
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
plotly.plotly.sign_in('spersad', 'oNkuP1yzbpN734Ag8M9P')
import plotly.graph_objs as go
from IPython.core.display import display, HTML

In [32]:
style.use('fivethirtyeight')
not_location_file = 'PATIENT_DATA_ALL_4.csv'
location_file = 'PATIENT_DATA_LOCATION.csv'
not_location = pd.read_csv(not_location_file, sep='\t')
location = pd.read_csv(location_file, sep='\t')
cna = pd.read_csv('data_CNA.xls.txt',sep='\t')

In [33]:
cna.set_index('Hugo_Symbol', inplace=True)
cna.drop(['Entrez_Gene_Id'], axis=1,inplace=True)
cna = cna.transpose()
cna.sort_index(inplace=True)
#cna = cna[:-1] # getting rid of the last row

#merged.sort_values('SAMPLE_ID', ascending=False)
not_location.set_index('SAMPLE_ID',inplace=True)
not_location.drop('PATIENT_ID', axis=1,inplace=True)
not_location.drop(['Unnamed: 0'], axis=1,inplace=True)
not_location.sort_index(inplace=True)

location.drop('PATIENT_ID', axis=1,inplace=True)
location.drop(['Unnamed: 0'], axis=1,inplace=True)
location.set_index('SAMPLE_ID', inplace=True)
location.sort_index(inplace=True)

clinical_n_cna = pd.concat([not_location, cna], axis=1)

columns = list(location.columns)
categories = list(range(len(columns)))
category_dict = dict(zip(columns, categories))

clinical_n_cna = clinical_n_cna.dropna(axis=0, how='any')

In [34]:
X = clinical_n_cna.as_matrix() 
Y = location.as_matrix()

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import time

In [27]:
randomize = True
if randomize:
    random_indices = np.random.permutation(len(X))
    # Randomized data
    X_r = X[random_indices]
    Y_r = Y[random_indices]

# Select a training, test and validation set
n_train = int(0.5*X.shape[0])
n_val = int(0.1*X.shape[0])

X_train, y_train = X_r[:n_train], Y_r[:n_train]
X_val, y_val = X_r[n_train:n_train+n_val], Y_r[n_train:n_train+n_val]
X_test, y_test = X_r[n_train+n_val:], Y_r[n_train+n_val:]

In [28]:
# Turn down for faster run time
n_samples = 10000

penalty = 'l1'

train_samples, n_features = X_train.shape
n_classes = np.unique(Y).shape[0]

t0 = time.clock()

print('Dataset 2CN, train_samples=%i, n_features=%i, n_classes=%i'
      % (train_samples, n_features, n_classes))

models = {'ovr': {'name': 'One versus Rest', 'iters': [1, 3, 7, 11, 13, 15]},
          'multinomial': {'name': 'Multinomial', 'iters': [1, 3, 7, 11, 13, 15]}}

# Solver which handles multiclass and l1 penalty
solver = 'newton-cg'

if penalty == 'l1':
    print('Using l1 regression')
    models = {'ovr': {'name': 'One versus Rest', 'iters': [1, 3]}}
    solver = 'liblinear'
    
for model in models:
    # Add initial chance-level values for plotting purpose
    test_accuracies = [1 / n_classes]
    tr_accuracies = [1/n_classes]
    val_accuracies = [1/n_classes]
    times = [0]
    densities = [1]

    model_params = models[model]

    # Small number of epochs for fast runtime
    for this_max_iter in model_params['iters']:
        print('[model=%s, solver=%s] Number of epochs: %s' %
              (model_params['name'], solver, this_max_iter))
        #lr = LogisticRegression(solver=solver,multi_class=model,C=1,penalty=penalty,fit_intercept=True, max_iter=this_max_iter, random_state=42)
        max_depth = 30
        lr = MultiOutputRegressor(RandomForestRegressor(max_depth=max_depth,
                                                          random_state=0))
        t1 = time.clock()
        lr.fit(X_train, y_train)
        train_time = time.clock() - t1

        y_pred = lr.predict(X_test)
        accuracy = np.sum(y_pred == y_test) / y_test.shape[0]
        #density = np.mean(lr.get_params(deep=True) != 0, axis=1) * 100
        tr_accuracy = np.sum(y_train == lr.predict(X_train)) / y_train.shape[0]
        val_accuracy = np.sum(y_val == lr.predict(X_val)) / y_val.shape[0]
        
        test_accuracies.append(accuracy)
        #densities.append(density)
        tr_accuracies.append(tr_accuracy)
        val_accuracies.append(val_accuracy)
        times.append(train_time)
    models[model]['times'] = times
    #models[model]['densities'] = densities
    models[model]['test_accuracies'] = test_accuracies
    models[model]['tr_accuracies'] = tr_accuracies
    models[model]['val_accuracies'] = val_accuracies
    
    print('Test accuracy for model %s: %.4f' % (model, test_accuracies[-1]))
    print('Train accuracy for model %s: %.4f' % (model, tr_accuracies[-1]))
    print('Validation accuracy for model %s: %.4f' % (model, val_accuracies[-1]))
    #print('%% non-zero coefficients for model %s, '
    #     'per class:\n %s' % (model, densities[-1]))
    print('Run time (%i epochs) for model %s:'
          '%.2f' % (model_params['iters'][-1], model, times[-1]))

fig = plt.figure()
ax = fig.add_subplot(111)

for model in models:
    data = []
    for accuracy in ['tr_accuracies', 'val_accuracies','test_accuracies']:
        
        trace = go.Scatter(
                            x = models[model]['times'],
                            y = models[model][accuracy],
                            name = accuracy
                        )

        data.append(trace)
    layout = go.Layout(
        title = 'Accuracy for {0} model'.format(model),
        xaxis = dict(title = 'Time'),
        yaxis = dict(title = 'Accuracy'),
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
        
run_time = time.clock() - t0
print('Example run in %.3f s' % run_time)

Dataset 2CN, train_samples=51, n_features=23184, n_classes=3
Using l1 regression
[model=One versus Rest, solver=liblinear] Number of epochs: 1
[model=One versus Rest, solver=liblinear] Number of epochs: 3
Test accuracy for model ovr: 17.9756
Train accuracy for model ovr: 21.5098
Validation accuracy for model ovr: 18.7000
Run time (3 epochs) for model ovr:4.64


Example run in 9.815 s


In [29]:
def run_regression(X_r, Y_r, penalty = 'l2', per_train=0.4, per_val=0.15):
    
    # Select a training, test and validation set

    n_train = int(per_train*X_r.shape[0])
    n_val = int(per_val*X_r.shape[0])


    X_test, y_test = X_r[:n_train], Y_r[:n_train]
    X_val, y_val = X_r[n_train:n_train+n_val], Y_r[n_train:n_train+n_val]
    X_train, y_train = X_r[n_train+n_val:], Y_r[n_train+n_val:]

    # How many of each cancer type are in the train, test and validations set?
    tr_count = y_train.value_counts()/np.sum( y_train.value_counts())
    val_count = y_val.value_counts()/np.sum(y_val.value_counts())
    te_count = y_test.value_counts()/np.sum(y_test.value_counts())


    # Plot the distribution of test, train and val points. Will throw a KeyError if any
    try:
        trace1 = go.Bar(
            x=['BREAST','IDC', 'MDLC', 'ILC'],
            y=[tr_count[i] for i in range(4)],
            name='Training Set'
        )
        trace2 = go.Bar(
            x=['BREAST','IDC', 'MDLC', 'ILC'],
            y=[val_count[i] for i in range(4)],
            name='Validation Set'
        )

        trace3 = go.Bar(
            x=['BREAST','IDC', 'MDLC', 'ILC'],
            y=[te_count[i] for i in range(4)],
            name='Test Set'
        )

        data = [trace1, trace2, trace3]
        layout = go.Layout(
            barmode='group',
            title = 'Distribution of Categories over Data Sets',
            xaxis = dict(title = 'Categories'),
            yaxis = dict(title = 'Percentage of Data Set'),
        )

        fig = go.Figure(data=data, layout=layout)
        iplot(fig, filename='grouped-bar')

    except:
        print(tr_count)
        print(val_count)
        print(te_count)
        print('One of the data sets does not contain all the categories!')
        assert False

    # plot the training data in two dimensions
    performPCA(X_train, y_train)
    
    t0 = time.clock()

    train_samples, n_features = X_train.shape
    n_classes = np.unique(Y).shape[0]

    print('Dataset 2CN, train_samples=%i, n_features=%i, n_classes=%i'
          % (train_samples, n_features, n_classes))

    models = {'ovr': {'name': 'One versus Rest', 'iters': [1, 3, 7, 11, 13, 15, 17, 21]},
              'multinomial': {'name': 'Multinomial', 'iters': [1, 3, 7, 11, 13, 15, 17, 21]}}

    # Solver which handles multiclass and l1 penalty
    solver = 'newton-cg'

    if penalty == 'l1':
        print('Using l1 regression')
        models = {'ovr': {'name': 'One versus Rest', 'iters': [1, 3, 7, 11, 13, 15, 17, 21]}}
        solver = 'liblinear'

    best_val_accuracy = 0
    best_model = None 
    
    def get_acc(y_true, y_pred):
        #F1 score
        from sklearn.metrics import f1_score
        return f1_score(y_true, y_pred)
    
        #accuracy
        return np.sum(y_true == y_pred) / y_true.shape[0]
        return 
    
    
    for model in models:
        # Add initial chance-level values for plotting purpose
        test_accuracies = [1 / n_classes]
        tr_accuracies = [1/n_classes]
        val_accuracies = [1/n_classes]
        times = [0]
        densities = [1]

        model_params = models[model]

        # Small number of epochs for fast runtime
        for this_max_iter in model_params['iters']:
            print('[model=%s, solver=%s] Number of epochs: %s' %
                  (model_params['name'], solver, this_max_iter))
            lr = LogisticRegression(solver=solver,
                                    multi_class=model,
                                    C=1,
                                    penalty=penalty,
                                    fit_intercept=True,
                                    max_iter=this_max_iter,
                                    random_state=42,
                                    )
            
            lr.fit(X_train, y_train)
            
            accuracy = get_acc(y_test,lr.predict(X_test))
            density = np.mean(lr.coef_ != 0, axis=1) * 100
            tr_accuracy = get_acc(y_train,lr.predict(X_train))
            val_accuracy = get_acc(y_val, lr.predict(X_val))

            test_accuracies.append(accuracy)
            densities.append(density)
            tr_accuracies.append(tr_accuracy)
            val_accuracies.append(val_accuracy)
            times.append(this_max_iter)
            
            if accuracy>best_val_accuracy:
                best_model = lr
                
        models[model]['times'] = times
        models[model]['densities'] = densities
        models[model]['test_accuracies'] = test_accuracies
        models[model]['tr_accuracies'] = tr_accuracies
        models[model]['val_accuracies'] = val_accuracies

        print('Test accuracy for model %s: %.4f' % (model, test_accuracies[-1]))
        print('Train accuracy for model %s: %.4f' % (model, tr_accuracies[-1]))
        print('Validation accuracy for model %s: %.4f' % (model, val_accuracies[-1]))
        print('%% non-zero coefficients for model %s, '
              'per class:\n %s' % (model, densities[-1]))
        print('Run time (%i epochs) for model %s:'
              '%.2f' % (model_params['iters'][-1], model, times[-1]))

    fig = plt.figure()
    ax = fig.add_subplot(111)

    for model in models:
        data = []
        for accuracy in ['tr_accuracies', 'val_accuracies','test_accuracies']:
            trace = go.Scatter(
                                x = models[model]['times'],
                                y = models[model][accuracy],
                                name = accuracy
                            )

            data.append(trace)
        layout = go.Layout(
            title = 'Accuracy for {0} model'.format(model),
            xaxis = dict(title = 'Iterations'),
            yaxis = dict(title = 'Accuracy'),
        )

        fig = go.Figure(data=data, layout=layout)
        iplot(fig)

    run_time = time.clock() - t0
    print('Example run in %.3f s' % run_time)

    from sklearn.metrics import confusion_matrix
    c = confusion_matrix(y_test, best_model.predict(X_test))

    trace = go.Heatmap(z=(c.T/np.sum(c,axis=1)).T,                   
                       x=['BREAST','IDC', 'MDLC', 'ILC'],
                       y=['BREAST','IDC', 'MDLC', 'ILC'])
    data=[trace]

    layout = go.Layout(
        title = 'Confusion Matrix for Test Predictions',
        xaxis = dict(title = 'Predicted'),
        yaxis = dict(title = 'Truth'),
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    
    c = confusion_matrix(y_train, best_model.predict(X_train))
    trace = go.Heatmap(z=(c.T/np.sum(c,axis=1)).T,                   
                       x=['BREAST','IDC', 'MDLC', 'ILC'],
                       y=['BREAST','IDC', 'MDLC', 'ILC'])
    data=[trace]

    layout = go.Layout(
        title = 'Confusion Matrix for Training Predictions',
        xaxis = dict(title = 'Predicted'),
        yaxis = dict(title = 'Truth'),
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)


    return best_model, lr